In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#new terminal for jupyter notebook to display console outputs
%qtconsole

In [4]:
%cd yolov5
!python detect.py --weights ./runs/train/exp2/weights/best.pt --source ../random-cars-dataset/images/ --save-txt --exist-ok --save-conf
%cd ..

C:\Users\HP\Desktop\ML\ANPR\yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=True, img_size=640, iou_thres=0.45, name='exp', nosave=False, project='runs/detect', save_conf=True, save_txt=True, source='../random-cars-dataset/images/', update=False, view_img=False, weights=['./runs/train/exp2/weights/best.pt'])
Fusing layers... 
image 1/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars0.png: 352x640 1 license_plate, Done. (0.542s)
image 2/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars1.png: 416x640 1 license_plate, Done. (0.304s)
image 3/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars102.png: 512x640 1 license_plate, Done. (0.385s)
image 4/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars107.png: 384x640 1 license_plate, Done. (0.269s)
image 5/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars108.png: 512x640 1 licens

YOLOv5  80d7d62 torch 1.8.1+cpu CPU

Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPS


In [5]:
PATH='./random-cars-dataset'

def decode_yolo_format(img_name,width,height):
    
    img_result_path=f'./yolov5/runs/detect/exp/labels/{img_name}.txt'
    
    #creates a file if not exists
    if not os.path.exists(img_result_path):
        with open(img_result_path, 'w'): pass
        
    label_file=open(img_result_path)
    
    labels=label_file.readlines()
    f=open(f'{PATH}/bounding_boxes_pred/{img_name}.txt','w')
    
    for label in labels:
        s=label.replace('\n','').split(' ')
        s=[ float(i) for i in s]

        bottom_x=((2*s[1]+s[3])/2)
        top_x=bottom_x-s[3]

        bottom_y=((2*s[2]+s[4])/2)
        top_y=bottom_y-s[4]

        top_x=int(top_x*width)
        bottom_x=int(bottom_x*width)

        top_y=int(top_y*height)
        bottom_y=int(bottom_y*height)

        conf=s[5]
        
        f.write(f'0 {conf} {top_x} {top_y} {bottom_x} {bottom_y} \n')
    
    f.close()
    label_file.close()


In [6]:
import os
import xml.etree.ElementTree as ET
import re

def xml_to_txt(file):    
    
    #reading xml file
    root=ET.parse(f'{PATH}/annotations/{file}.xml').getroot()

    #extracting bounding box
    ob=root.find('object')

    xmin=int(ob.find('bndbox/xmin').text)
    ymin=int(ob.find('bndbox/ymin').text)
    xmax=int(ob.find('bndbox/xmax').text)
    ymax=int(ob.find('bndbox/ymax').text)
    
    
    #writing to text file
    f=open(f'{PATH}/bounding_boxes/{file}.txt','w')
    f.write(f'0 {xmin} {ymin} {xmax} {ymax}')
    f.close()

In [17]:
import os
def clear_dir(dir):
    
    for f in os.listdir(dir):
        os.remove(os.path.join(dir,f))
        
#clearing dirs
clear_dir(f'{PATH}/bounding_boxes/')
clear_dir(f'{PATH}/bounding_boxes_pred/')

In [18]:
import os
try:
 from PIL import Image
except ImportError:
 import Image


#extracting bounding box of each image to calculate mAP of yolo
for img in os.listdir(f'{PATH}/images'):
    
    #extracting file name, eg: Car1 from Car1.png
    f_name=img.split('.')[0]
    
    #getting ground truth
    xml_to_txt(f_name)
    
    #get image size
    img_path=f'{PATH}/images/{img}'
    width,height=Image.open(img_path).size
    
    
    #getting predicting bounding box
    decode_yolo_format(f_name,width,height)

In [39]:
!git clone https://github.com/Cartucho/mAP

Cloning into 'mAP'...


In [40]:
#Steps to evalucate mAP

# 1. Create the ground-truth files
# 2. Copy the ground-truth files into the folder input/ground-truth/
# 3. Create the detection-results files
# 4. Copy the detection-results files into the folder input/detection-results/
# 5. Run the code: python main.py

In [41]:
#Step 1 and 3 are done, now we need to copy results to the above mentioned folders

In [20]:
import shutil
import os
    
#move all files from one directory to another
def move_all_files(source_dir,target_dir):

    file_names = os.listdir(source_dir)

    for file_name in file_names:
        shutil.copy(os.path.join(source_dir, file_name), target_dir)

In [21]:
clear_dir('./mAP/input/ground-truth/')
clear_dir('./mAP/input/detection-results/')
clear_dir('./mAP/input/images-optional/')

#moving ground truth files
move_all_files('./random-cars-dataset/bounding_boxes/','./mAP/input/ground-truth/')

#moving yolo predictions
move_all_files('./random-cars-dataset/bounding_boxes_pred/','./mAP/input/detection-results/')

#moving images(optional)
move_all_files('./random-cars-dataset/images/','./mAP/input/images-optional/')

In [23]:
!python ./mAP/main.py

48.78% = 0 AP 
mAP = 48.78%


Traceback (most recent call last):
  File "./mAP/main.py", line 753, in <module>
cv2.error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgcodecs\src\loadsave.cpp:753: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

